# Notebook to build PP-PicoDet model with basic pytorch building blocks
- https://arxiv.org/abs/2111.00902

## Note new version picodetv2
- https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.4/ppdet/modeling/architectures/picodet.py
- LCNET backbone (https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.4/ppdet/modeling/backbones/lcnet.py)
- LCPAN (https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.4/ppdet/modeling/necks/lc_pan.py)
- picoheadv2 (https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.4/ppdet/modeling/heads/pico_head.py)

In [4]:
import torch
import torch.nn as nn

# Backbone PPLCNET (https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.4/ppdet/modeling/backbones/lcnet.py)

In [36]:
# utils 
NET_CONFIG = {
    "blocks2":
    #k, in_c, out_c, s, use_se
    [[3, 16, 32, 1, False], ],
    "blocks3": [
        [3, 32, 64, 2, False],
        [3, 64, 64, 1, False],
    ],
    "blocks4": [
        [3, 64, 128, 2, False],
        [3, 128, 128, 1, False],
    ],
    "blocks5": [
        [3, 128, 256, 2, False],
        [5, 256, 256, 1, False],
        [5, 256, 256, 1, False],
        [5, 256, 256, 1, False],
        [5, 256, 256, 1, False],
        [5, 256, 256, 1, False],
    ],
    "blocks6": [[5, 256, 512, 2, True], [5, 512, 512, 1, True]]
}



def make_divisible(v, divisor=8, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

class ConvBNLayer(nn.Module):
    def __init__(self,
                 num_channels,
                 filter_size,
                 num_filters,
                 stride,
                 num_groups=1):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channels=num_channels,
                              out_channels=num_filters,
                              kernel_size=filter_size,
                              stride=stride,
                              padding=(filter_size-1)//2,
                              groups=num_groups,
                              bias=False)
        # in inference fuse to conv
        self.bn = nn.BatchNorm2d(num_filters)
        self.hardswish = nn.Hardswish()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.hardswish(x)
        return x

class SEModule(nn.Module):
    def __init__(self, channel, reduction=4):
        super().__init__()
        self.avg_pool= nn.AdaptiveAvgPool2d(1)
        self.conv1 = nn.Conv2d(in_channels=channel,
                               out_channels=channel//reduction,
                               kernel_size=1,
                               stride=1,
                               padding=0)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=channel//reduction,
                               out_channels=channel,
                               kernel_size=1,
                               stride=1,
                               padding=0)
        self.hardsigmoid = nn.Hardsigmoid()
    def forward(self, x):
        idendity = x
        x = self.avg_pool(x)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.hardsigmoid(x)
        x = x*idendity
        return x
        
        
class DepthWiseSeparable(nn.Module):
    def __init__(self,
                 num_channels,
                 num_filters,
                 stride,
                 dw_size=3,
                 use_se=False):
        super().__init__()
        self.use_se = use_se
        self.dw_conv = ConvBNLayer(num_channels=num_channels,
                                   num_filters=num_channels,
                                   filter_size=dw_size,
                                   stride=stride,
                                   num_groups=num_channels)
        if use_se:
            self.se = SEModule(num_channels)
        self.pw_conv = ConvBNLayer(num_channels=num_channels,
                                   filter_size=1,
                                   num_filters=num_filters,
                                   stride=1)
    def forward(self, x):
        x = self.dw_conv(x)
        if self.use_se:
            x = self.se(x)
        x = self.pw_conv(x)
        return x
            
        
        

test_scale = 1
conv_bn = ConvBNLayer(num_channels=3,
                      filter_size=3,
                      num_filters=make_divisible(16*test_scale),
                      stride=2)
se_module = SEModule(make_divisible(16*test_scale))
blocks2 = nn.Sequential(*[DepthWiseSeparable(
                            num_channels=make_divisible(in_c*test_scale),
                            num_filters=make_divisible(out_c*test_scale),
                            dw_size=k,
                            stride=s,
                            use_se=se) for i, (k, in_c, out_c, s, se) in enumerate(NET_CONFIG["blocks2"])])

test_input = torch.randn(1, 3, 320, 320)
test_out = conv_bn(test_input)
test_out = blocks2(test_out)
print(test_out.size())

torch.Size([1, 32, 160, 160])


In [141]:
class LCNet(nn.Module):
    def __init__(self, scale=1.0, feature_maps=[3, 4, 5]):
        super().__init__()
        self.scale = scale
        self.feature_maps = feature_maps
        
        out_channels = []
        self.conv1 = ConvBNLayer(num_channels=3,
                                 filter_size=3,
                                 num_filters=make_divisible(16*scale),
                                 stride=2)
        self.blocks2 = nn.Sequential(*[DepthWiseSeparable(
                            num_channels=make_divisible(in_c*scale),
                            num_filters=make_divisible(out_c*scale),
                            dw_size=k,
                            stride=s,
                            use_se=se) for i, (k, in_c, out_c, s, se) in enumerate(NET_CONFIG["blocks2"])])
        self.blocks3 = nn.Sequential(*[DepthWiseSeparable(
                            num_channels=make_divisible(in_c*scale),
                            num_filters=make_divisible(out_c*scale),
                            dw_size=k,
                            stride=s,
                            use_se=se) for i, (k, in_c, out_c, s, se) in enumerate(NET_CONFIG["blocks3"])])
        
        out_channels.append(make_divisible(NET_CONFIG["blocks3"][-1][2]*scale))
        
        self.blocks4 = nn.Sequential(*[DepthWiseSeparable(
                            num_channels=make_divisible(in_c*scale),
                            num_filters=make_divisible(out_c*scale),
                            dw_size=k,
                            stride=s,
                            use_se=se) for i, (k, in_c, out_c, s, se) in enumerate(NET_CONFIG["blocks4"])])
        
        out_channels.append(make_divisible(NET_CONFIG["blocks4"][-1][2]*scale))
        
        self.blocks5 = nn.Sequential(*[DepthWiseSeparable(
                            num_channels=make_divisible(in_c*scale),
                            num_filters=make_divisible(out_c*scale),
                            dw_size=k,
                            stride=s,
                            use_se=se) for i, (k, in_c, out_c, s, se) in enumerate(NET_CONFIG["blocks5"])])
        
        out_channels.append(make_divisible(NET_CONFIG["blocks5"][-1][2]*scale))
        
        self.blocks6 = nn.Sequential(*[DepthWiseSeparable(
                            num_channels=make_divisible(in_c*scale),
                            num_filters=make_divisible(out_c*scale),
                            dw_size=k,
                            stride=s,
                            use_se=se) for i, (k, in_c, out_c, s, se) in enumerate(NET_CONFIG["blocks6"])])
        
        out_channels.append(make_divisible(NET_CONFIG["blocks6"][-1][2]*scale))
        self._out_channels = [
            ch for idx, ch in enumerate(out_channels) if idx+2 in feature_maps]
        
    def forward(self, inputs):
        x = inputs['image']
        outs = []
        
        x = self.conv1(x)
        x = self.blocks2(x)
        x = self.blocks3(x)
        outs.append(x)
        x = self.blocks4(x)
        outs.append(x)
        x = self.blocks5(x)
        outs.append(x)
        x = self.blocks6(x)
        outs.append(x)
        outs = [o for i, o in enumerate(outs) if i+2 in self.feature_maps]
        return outs

backbone = LCNet()
inp_t = torch.randn(1, 3, 320, 320)
output = backbone({'image': inp_t})
for t in output:
    print(t.size())

torch.Size([1, 128, 40, 40])
torch.Size([1, 256, 20, 20])
torch.Size([1, 512, 10, 10])


# Detector Neck LCPan: (https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.4/ppdet/modeling/necks/lc_pan.py